In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy  as np
import matplotlib.pyplot as plt

In [ ]:
import os
base_dir = '../input/covid19/Data'
train_dir = os.path.join(base_dir,'train')
test_dir = os.path.join(base_dir,'test')



train_covid_dir = os.path.join(train_dir,'COVID19')
train_normal_dir = os.path.join(train_dir,'NORMAL')


test_covid_dir = os.path.join(test_dir,'COVID19')
test_normal_dir = os.path.join(test_dir,'NORMAL')


In [ ]:
train_covid_names = os.listdir(train_covid_dir)
print(train_covid_names[:10])

train_normal_names = os.listdir(train_normal_dir)
print(train_normal_names[:10])

test_covid_names = os.listdir(test_covid_dir)
print(test_covid_names[:10])

test_normal_names = os.listdir(test_normal_dir)
print(test_normal_names[:10])

In [ ]:
print("Train Dataset Covid Images:",len(train_covid_names))
print("Train Dataset Normal Images:",len(train_normal_names))
print("Test Dataset Covid Images:",len(test_covid_names))
print("Test Dataset Normal Images:",len(test_normal_names))
print("Total Train Images:",len(train_covid_names+train_normal_names))
print("Total Test Images:",len(test_covid_names+test_normal_names))

In [ ]:
import matplotlib.image as mpimg

nrows = 4
ncols = 4

fig=plt.gcf()
fig.set_size_inches(12,12)

next_covid_pic = [os.path.join(train_covid_dir,fname)for fname in train_covid_names[0:8]]
next_normal_pic = [os.path.join(train_normal_dir,fname)for fname in train_normal_names[0:8]]

print(next_covid_pic)
print(next_normal_pic)

for i , img_path in enumerate(next_covid_pic+next_normal_pic):
    data = img_path.split('/',6)[6]
    sp =plt.subplot(nrows,ncols,i+1)
    sp.axis('Off')
    img = mpimg.imread(img_path)
    sp.set_title(data,fontsize=10)
    plt.imshow(img,cmap='gray')
    
plt.show()    

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size =(150,150),
                                                   subset = 'training',
                                                   batch_size = 32,
                                                   class_mode = 'binary')

validation_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size =(150,150),
                                                        subset = 'validation',
                                                        batch_size = 32,
                                                        class_mode = 'binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                 target_size =(150,150),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

In [ ]:
train_generator.class_indices

In [ ]:
train_generator.image_shape

In [ ]:
model = Sequential()

model.add(Conv2D(32,(5,5),padding='SAME',activation='relu',input_shape=(150,150,3)))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.5))

model.add(Conv2D(64,(5,5),padding='SAME',activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256,activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))

model.summary()

In [ ]:
#compile the model

from tensorflow.keras.optimizers import Adam
model.compile(Adam(lr = 0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#training the Model

history = model.fit(train_generator,
                   epochs=30,
                   validation_data = validation_generator,
                   validation_steps = 10)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training','validation'])
plt.title('Training and Validation Accuracy')
plt.xlabel('epoch')

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('test acc:{} test loss:{}'.format(test_acc,test_loss))